# DEEP LEARNING 

# Partie sur les images

### Objectif à dépasser : weighted F1-score = 0.5534 (Resnet)
avec CNN/Transfer learning

Import libraries:

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images')

In [9]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Récupérer les données:

In [10]:
X = pd.read_csv('X_train_update.csv',index_col=0)
y = pd.read_csv('Y_train_CVw08PX.csv',index_col=0)

Liste des images:

In [11]:
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds')

In [12]:
import os #Miscellaneous operating system interfaces
#https://docs.python.org/3/library/os.html

#get current working directory
current_path = os.getcwd() 

#Training images path
images_path = current_path + r'/images/image_train/'

#List with the name of all training images
images_list = os.listdir(images_path)

Géneration nom des images:

In [13]:
#Create a column with the name of the picture
X['image name'] = 'image_' + X['imageid'].map(str) + '_product_' + X['productid'].map(str) + '.jpg'
X['image name']

0        image_1263597046_product_3804725264.jpg
1         image_1008141237_product_436067568.jpg
2          image_938777978_product_201115110.jpg
3           image_457047496_product_50418756.jpg
4         image_1077757786_product_278535884.jpg
                          ...                   
84911      image_941495734_product_206719094.jpg
84912    image_1188462883_product_3065095706.jpg
84913     image_1009325617_product_440707564.jpg
84914    image_1267353403_product_3942400296.jpg
84915       image_684671297_product_57203227.jpg
Name: image name, Length: 84916, dtype: object

### Répartition des images dans les échantillons train, validation et test

Répartion :

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

X_val = X_train.iloc[:1600,:]
y_val = y_train.iloc[:1600,:]

X_train = X_train.iloc[:14400,:]
y_train = y_train.iloc[:14400:,:]

X_test = X_test.iloc[:4000,:]
y_test = y_test.iloc[:4000,:]

Qualité de la répartitioon

trainy = pd.DataFrame(y_train.value_counts(), columns = ['Nombre_images_train'])
testy = pd.DataFrame(y_test.value_counts(), columns = ['Nombre_images_test'])
train_test = trainy.merge(testy, right_index = True, left_index = True)
train_test['y'] = train_test.index
yval = []
for i in train_test['y']:
    yval.append(i[0])
train_test['y'] = yval
train_test

In [15]:
X_train["class"] = y_train
X_test["class"] = y_test
X_val["class"] = y_val

# TRANSFER LEARNING

Image Generator :

In [16]:
# Creating objects for image augmentations


from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale = 1./255,  
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

val_data_generator = ImageDataGenerator(rescale = 1./255,
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

test_data_generator = ImageDataGenerator(rescale = 1./255)

In [17]:
batch_size = 32
path = os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images\\image_train')
X_train["class"] = X_train["class"].astype(str)
X_test["class"] = X_test["class"].astype(str)
X_val["class"] = X_val["class"].astype(str)

train_generator = train_data_generator.flow_from_dataframe(dataframe=X_train,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size)


val_generator = val_data_generator.flow_from_dataframe(dataframe=X_val,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size)

test_generator = test_data_generator.flow_from_dataframe(dataframe=X_test,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size)

Found 14400 validated image filenames belonging to 27 classes.
Found 1600 validated image filenames belonging to 27 classes.
Found 4000 validated image filenames belonging to 27 classes.


Model creation :

In [18]:
pip install -U efficientnet

Note: you may need to restart the kernel to use updated packages.


In [24]:
import efficientnet.keras as efn
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout,Dense, Flatten, GlobalAveragePooling2D

In [27]:
model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

    
n_class = 27
 
model = Sequential()
model.add(efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet'))
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation="sigmoid"))

from tensorflow.keras.optimizers import RMSprop

    
# Compiling the model
model.compile(RMSprop(lr=0.0001, decay=1e-6),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Functional) (None, 7, 7, 1280)        4049564   
_________________________________________________________________
flatten_3 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              64226304  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 27)                27675     
Total params: 68,303,543
Trainable params: 68,261,527
Non-trainable params: 42,016
_________________________________________________________________


Training : 

In [ ]:
# We will try to train the last stage of ResNet50


# Training the model for 10 epochs
model.fit_generator(train_generator, 
                                epochs = 2,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data = val_generator,
                                validation_steps = len(X_val)//batch_size)


C:\Users\barry\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
450/450 [==============================] - 2903s 6s/step - loss: 3.2824 - accuracy: 0.1920 - val_loss: 2.3432 - val_accuracy: 0.3331
Epoch 2/2
209/450 [============>.................] - ETA: 24:39 - loss: 2.5095 - accuracy: 0.3062

Entrainement 4 derniers layers freezed :

In [ ]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
tl_2_defreeze = model.fit_generator(generator=train_generator, 
                                epochs = 2,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data=test_generator,
                                validation_steps=len(X_val)//batch_size
                                )

In [ ]:
y_tl_pred_2 = model.predict_generator(test_generator, verbose = 1)
y_tl_pred_2 = y_tl_pred_2.argmax(axis = 1)

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_generator.classes, y_tl_pred_2 , average = "weighted")

In [ ]:
from sklearn.metrics import confusion_matrix
print(pd.DataFrame(confusion_matrix( test_generator.classes, y_tl_pred_2)))